In [1]:
from collections import Counter
import torch.nn as nn
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [90]:
class MyLSTM(nn.Module):
    def __init__(self, output_size, hidden_size, n_layers=1):
        super(MyLSTM, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.n_layers = n_layers
        self.LSTM = nn.LSTM(self.output_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        self.Softmax = nn.Softmax(1)
    def forward(self,x,hidden):
        out, (h, c)  = self.LSTM(x,hidden)
        out = self.dropout(out)
        x = self.fc(out)
        x = self.Softmax(self.fc(out))
        return x, (h, c)

In [91]:
lr=0.0001
LSTM=MyLSTM(1603,32,input_size=1)
Loss = nn.CrossEntropyLoss()
optimizer = optim.AdamW(LSTM.parameters(), lr=lr)


In [125]:
def micro_batch(input,index,input_size,output_size):
    sting=input[:,index:index+input_size]
    label_ind=input[:,index+input_size:index+input_size+1]
    label_ind=label_ind.type(torch.int8)
    label=np.zeros([input.shape[0],output_size])
    for i in range(input.shape[0]):
        label[i][label_ind[i][0]]+=1
    label=torch.Tensor(label)
    return sting, label
    

In [ ]:
input = torch.Tensor([[46,12,95,78],[46,59,41,13]])
# input = torch.Tensor([[46,12],[46,59]])
for i in range(1000):
    h0 = torch.randn(1,  32)
    c0 = torch.randn(1,  32)
    optimizer.zero_grad()
    total_loss = torch.tensor([0.0], requires_grad=True)
    for i in range(input.shape[1]-2):
        sting, label=micro_batch(input,i,1,1603)
        output, (h0, c0) = LSTM(sting, (h0, c0)) 
        total_loss=total_loss+Loss(output,label)
    total_loss.backward()
    # Обновить веса
    optimizer.step()
    print(total_loss)